In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
import plotly.express as px
import seaborn as sns
from tkinter import *
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression,LogisticRegression,SGDRegressor
from sklearn.metrics import r2_score, confusion_matrix, ConfusionMatrixDisplay,accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Choose & Read the File
mainflag=True
df=None
flag=False
while flag==False:
    filetypes = (('csv files', '*.csv'),('Excel files', '*.xlsx'))
    filepath = fd.askopenfilename(title='Open files',initialdir='.',filetypes=filetypes)
    #showinfo(title='Selected Files',message=filepath)
    ad=filepath[filepath.find('.')+1:]
    if ad=='csv':
        #print('CSV File')
        df=pd.read_csv(filepath)
        flag=True
    elif ad=='xlsx':
        #print('Excel File')
        df=pd.read_excel(filepath)
        flag=True
    else:
        print('another type')
        flag=True
df

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0.0
1,15810944,Male,35.0,20000.0,0.0
2,15668575,Female,26.0,43000.0,0.0
3,15603246,Female,27.0,57000.0,0.0
4,15804002,Male,19.0,76000.0,0.0
...,...,...,...,...,...
397,15691863,Female,46.0,41000.0,1.0
398,15706071,Male,51.0,23000.0,1.0
399,15654296,Female,50.0,20000.0,1.0
400,15755018,Male,36.0,33000.0,0.0


In [3]:
delsel=[]
stdsel=[]
print('Select the study Columns')
for i in df.columns:
    print('Do you Select ',(i), ' for your study (Y/N)<Y>? ')
    sel = input()
    if sel=='n' or sel=='N':
        delsel.append(i)
    else:
        stdsel.append(i)

df=df.drop(delsel,axis=1)
print(df)
for i in df.columns:
    print(i,' is ',df[i].dtypes)


Select the study Columns
Do you Select  User ID  for your study (Y/N)<Y>? 
n
Do you Select  Gender  for your study (Y/N)<Y>? 

Do you Select  Age  for your study (Y/N)<Y>? 

Do you Select  EstimatedSalary  for your study (Y/N)<Y>? 

Do you Select  Purchased  for your study (Y/N)<Y>? 

     Gender   Age  EstimatedSalary  Purchased
0      Male  19.0          19000.0        0.0
1      Male  35.0          20000.0        0.0
2    Female  26.0          43000.0        0.0
3    Female  27.0          57000.0        0.0
4      Male  19.0          76000.0        0.0
..      ...   ...              ...        ...
397  Female  46.0          41000.0        1.0
398    Male  51.0          23000.0        1.0
399  Female  50.0          20000.0        1.0
400    Male  36.0          33000.0        0.0
401  Female  49.0          36000.0        1.0

[402 rows x 4 columns]
Gender  is  object
Age  is  float64
EstimatedSalary  is  float64
Purchased  is  float64


In [4]:
if df.duplicated().sum()!=0:
    print('Remove duplication ',df.duplicated().sum(),' Rows')
    df.drop_duplicates(keep='first', inplace=True)
    df.info()
else:
    print('There\'s no Duplicated rows')

Remove duplication  22  Rows
<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 401
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           378 non-null    object 
 1   Age              378 non-null    float64
 2   EstimatedSalary  378 non-null    float64
 3   Purchased        378 non-null    float64
dtypes: float64(3), object(1)
memory usage: 14.8+ KB


In [15]:
loop=True
while loop==True:
    print('Choose the Column number to be predict')
    for i in range(0,df.shape[1]):
        print(i,'-', df.columns[i])
    try:
        column_no=int(input())
        if column_no > df.shape[1]-1 or column_no < 0:
            raise
        column_prd=df.columns[column_no]
        print(column_prd)
        loop=False
    except:
        loop=True

Choose the Column number to be predict
0 - Gender
1 - Age
2 - EstimatedSalary
3 - Purchased
3
Purchased


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 401
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           380 non-null    int32  
 1   Age              380 non-null    float64
 2   EstimatedSalary  380 non-null    float64
 3   Purchased        380 non-null    int32  
dtypes: float64(2), int32(2)
memory usage: 11.9 KB


In [17]:
for i in range(0,df.shape[1]):
    print(df.columns[i],'has missing \t \t',df[df.columns[i]].isna().sum(),'\tValues')

Gender has missing 	 	 0 	Values
Age has missing 	 	 0 	Values
EstimatedSalary has missing 	 	 0 	Values
Purchased has missing 	 	 0 	Values


In [18]:
mean_imputer=SimpleImputer(strategy='mean',missing_values=np.nan)
median_imputer=SimpleImputer(strategy='median',missing_values=np.nan)
mode_imputer=SimpleImputer(strategy='most_frequent', missing_values=np.nan)
miss_imputer=SimpleImputer(strategy='constant', fill_value='Missing',missing_values=np.nan)

for i in range(0,df.shape[1]):
    if df[df.columns[i]].isna().sum()==0:
        print('Column ', df.columns[i], ' has no null value')
        print('-----------------------------------------')
    else:
        if df[df.columns[i]].dtypes in ['object', 'category','bool']:
            print('For Categrical Columns ', df.columns[i], '\nyou Want 1-Most Frequent , 2-New Class "Missingg", <enter Number: default 1-Most Frequent>: ')
            chosen=input()
            if chosen == '2':
                print(df.columns[i])
                print('Will Fill Categorial Column',df.columns[i],'Nulls with the value \'Missing\'')
                df[df.columns[i]]=miss_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                print('-----------------------------------------')
            else:
                print(df.columns[i])
                print('Will Fill Categorial Column',df.columns[i],'Nulls with the Most Frequent values')
                df[df.columns[i]]=mode_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                print('-----------------------------------------')
        
        elif df[df.columns[i]].dtypes in ['int64', 'float64']:
            if len(df[df.columns[i]].dropna().unique())==2:
                print('I expected ', df.columns[i],' it is categorial column \ndo you want change it (Y,N)<y>: ')
                cvrt=input()
                if cvrt!= 'N' or cvrt != 'n':
                    print('For Categrical Columns ', df.columns[i], '\nyou Want 1-Most Frequent , 2-New Class "Missingg", <enter Number: default 1-Most Frequent>: ')
                    chosen=input()
                    if chosen == '2':
                        print(df.columns[i])
                        print('Will Fill Categorial Column',df.columns[i],'Nulls with the value \'Missing\'')
                        df[df.columns[i]]=miss_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                    else:
                        print(df.columns[i])
                        print('Will Fill Categorial Column',df.columns[i],'Nulls with the Most Frequent values')
                        df[df.columns[i]]=mode_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                        df[df.columns[i]]=df[df.columns[i]].astype('object')
                        print('-----------------------------------------')
                        
            else:
                print('For Numerical Columns ', df.columns[i], '\nyou Want 1-Mean , 2-Median, 3-Mode <enter Number: default 1-Mean>: ')
                chosen=input()
                if chosen==1:
                    df[df.columns[i]]=mode_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                    print('-----------------------------------------')
                elif chosen==2:
                    df[df.columns[i]]=median_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                    print('-----------------------------------------')
                else:
                    df[df.columns[i]]=mean_imputer.fit_transform(df[df.columns[i]].values.reshape(-1,1))
                    print('-----------------------------------------')

Column  Gender  has no null value
-----------------------------------------
Column  Age  has no null value
-----------------------------------------
Column  EstimatedSalary  has no null value
-----------------------------------------
Column  Purchased  has no null value
-----------------------------------------


In [19]:
print(df.isna().sum())

Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64


In [20]:
print(df.info())
print('------------------------------')
print('duplicated\t\t',df.duplicated().sum())
print('------------------------------')
print('None Values')
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 401
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           380 non-null    int32  
 1   Age              380 non-null    float64
 2   EstimatedSalary  380 non-null    float64
 3   Purchased        380 non-null    int32  
dtypes: float64(2), int32(2)
memory usage: 11.9 KB
None
------------------------------
duplicated		 0
------------------------------
None Values
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64


In [21]:
print('Convert Categorial Columns to Numirical Column')
flag_many=False
for i in range(0,df.shape[1]):
    if df[df.columns[i]].dtypes in ['object', 'category','bool']:
      if len(df[df.columns[i]].unique()) == 2:
        df[df.columns[i]]=LabelEncoder().fit_transform(df[df.columns[i]])
      else:
        flag_many=True
    if flag_many==True:
        df=pd.get_dummies(df,drop_first=True)
df

Convert Categorial Columns to Numirical Column


,Gender,Age,EstimatedSalary,Purchased
0,1,19.0,19000.0,0
1,1,35.0,20000.0,0
2,0,26.0,43000.0,0
3,0,27.0,57000.0,0
4,1,19.0,76000.0,0
...,...,...,...,...
397,0,46.0,41000.0,1
398,1,51.0,23000.0,1
399,0,50.0,20000.0,1
400,1,36.0,33000.0,0


In [22]:
print('Predicted Column is ',column_prd)
x=df.drop(column_prd,axis=1)
x=StandardScaler().fit_transform(x)
y=df[column_prd]
print(x)
print(y)

Predicted Column is  Purchased
[[ 1.05409255 -1.76148708 -1.49635537]
 [ 1.05409255 -0.24540997 -1.4673491 ]
 [-0.9486833  -1.09820335 -0.80020481]
 ...
 [-0.9486833   1.17591233 -1.4673491 ]
 [ 1.05409255 -0.15065515 -1.09026755]
 [-0.9486833   1.08115751 -1.00324873]]
0      0
1      0
2      0
3      0
4      0
      ..
397    1
398    1
399    1
400    0
401    1
Name: Purchased, Length: 380, dtype: int32


In [24]:
def clasifyit(x_train,y_train,x_test,y_test):
    params={
    'max_iter':[1,2,3,4,5,10,20,50]
    }
    clf=LogisticRegression()
    gsclf=GridSearchCV(clf,params)
    #Train Data
    gsclf.fit(x_train,y_train)
    resultdf_train=pd.DataFrame(gsclf.cv_results_)
    
    model=gsclf.best_estimator_
    pred_train=model.predict(x_train)
    pred_test=model.predict(x_test)
    print('Logistic Regression\n-------------------------')
    print('R2 Score Train',r2_score(y_train,pred_train))
    print('R2 Score Test',r2_score(y_test,pred_test))
    print('--------------------------------------------------------------------------------------------------------------')
          
    params={
    'n_estimators':[10,20,50],
    'max_depth':[1,2,3,4,5]
    }
    clf=XGBClassifier()
    gsclf=GridSearchCV(clf,params)
    #Train Data
    gsclf.fit(x_train,y_train)
    resultdf_train=pd.DataFrame(gsclf.cv_results_)
    
    model=gsclf.best_estimator_
    pred_train=model.predict(x_train)
    pred_test=model.predict(x_test)
    print('XGB Classifier\n-------------------------')
    print('R2 Score Train',r2_score(y_train,pred_train))
    print('R2 Score Test',r2_score(y_test,pred_test))
    print('--------------------------------------------------------------------------------------------------------------')
    
    params={
    'n_estimators':[10,20,50],
    'max_depth':[1,2,3,4,5]
    }
    clf=RandomForestClassifier()
    gsclf=GridSearchCV(clf,params)
    #Train Data
    gsclf.fit(x_train,y_train)
    resultdf_train=pd.DataFrame(gsclf.cv_results_)
    model=gsclf.best_estimator_
    pred_train=model.predict(x_train)
    pred_test=model.predict(x_test)
    print('Random Forest Classifier\n-------------------------')
    print('R2 Score Train',r2_score(y_train,pred_train))
    print('R2 Score Test',r2_score(y_test,pred_test))
    print('--------------------------------------------------------------------------------------------------------------')
    
def regessit(x_train,y_train,x_test,y_test):
    mxc=MinMaxScaler()
    params={'max_iter':range(1,100)}
    lr=SGDRegressor()
    gsclf=RandomizedSearchCV(lr,params)
    gsclf.fit(x_train,y_train)
    resultdf_train=pd.DataFrame(gsclf.cv_results_)
    display(pd.DataFrame(resultdf_train))

    model=gsclf.best_estimator_
    pred_train=model.predict(x_train)
    pred_test=model.predict(x_test)
    print('SGD Regressor\n-------------------------')
    print('R2 Score Train',r2_score(y_train,pred_train))
    print('R2 Score Test',r2_score(y_test,pred_test))
    print('--------------------------------------------------------------------------------------------------------------') 
    
#main Application    
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,shuffle=True)
if len(df[column_prd].unique())==2:
    print('We will use Classification')
    print('---------------------------')
    clasifyit(x_train,y_train,x_test,y_test)
else:
    print('We will use Regression')
    print('------------------------')
    regessit(x_train,y_train,x_test,y_test)

We will use Classification
---------------------------
Logistic Regression
-------------------------
R2 Score Train 0.2307403936269915
R2 Score Test 0.4910714285714285
--------------------------------------------------------------------------------------------------------------
XGB Classifier
-------------------------
R2 Score Train 0.6296157450796627
R2 Score Test 0.6041666666666666
--------------------------------------------------------------------------------------------------------------
Random Forest Classifier
-------------------------
R2 Score Train 0.6153701968134957
R2 Score Test 0.7172619047619047
--------------------------------------------------------------------------------------------------------------
